## Language Modeling using Surnames Analysis - Japanese Surnames

---
### Hypothesis
---

To relate a name to nationality, check for the most common letter combinations/pronounciations within a specific language that makes it unique.

In this iteration we are going to:
* use naive bayes
* compare accuracy, recall, and precision for classifying Japanese names.

------


In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB # Multi Naive Bayes with discrete values
from sklearn.feature_extraction.text import CountVectorizer # tokenize texts/build vocab
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

---
### Let's perform some EDA

---

In [ ]:
# read the csv file into data frame.
surname_csv = "data_set/surnames-dev.csv"

surname_df = pd.read_csv(surname_csv, index_col = None)

surname_df.shape

In [ ]:
# check columns
surname_df.columns

In [ ]:
# a glimpse at the data
surname_df.head()

In [ ]:
# rename the columns
surname_df.rename(columns = {'Unnamed: 0':'surname', 'Unnamed: 1':'nationality'}, inplace = True)

In [ ]:
surname_df.head()

In [ ]:
# types of nationalities
surname_df.nationality.unique()

In [ ]:
# there are 18 unique nationalities in the dataset
len(surname_df.nationality.unique())

In [ ]:
# Number of names per nationality
surname_df.groupby("nationality")["surname"].size()

#### Features Exploration

In [ ]:
features = surname_df["surname"] # features (x) needed to predict nationatlity
target = surname_df["nationality"] # what we are predicting (y)

---
### Train/Test Data

---
To make the data a little more accurate in it's predictions, we are going to split the surnames into train (65%) and test (35%) datasets.

In [ ]:
# vectorize features
cv = CountVectorizer()
X = cv.fit_transform(features)

In [ ]:
# list names in the feature
# cv.get_feature_names()

In [ ]:
# split the data to train the model
x_train, x_test, y_train, y_test = train_test_split(X, target, test_size=0.35, random_state = 32)

---
### Model Developement - Entire Model

---
Multinomial NB - entire model

In [ ]:
# fit the model
surname_model = MultinomialNB()
surname_model.fit(x_train, y_train)

In [ ]:
y_pred = surname_model.predict(x_test)

#### -Accuracy-
------
Note: Accuracy will be computed using sklearn.metrics's ```f1_score``` library.


------

In [ ]:
# Accuracy using test data
surname_accuracy = f1_score(y_test, y_pred, average='micro')
print(f"f1 score: {surname_accuracy}")

__Observation 1__: Accuracy  is at 50%.
Possible Reasons: The amount of data used to train the model is too low or it's because there is only a small amount of data.

In [ ]:
# Which nationalities is the model more accurate with
surname_null_accuracy = y_test.value_counts().head(5) / len(y_test)
print(surname_null_accuracy)

__Observation 2__: This model is more accurate with Russian names. Japanese surnames accuracy is at 5% compared to Russia's 48%. This could be due to Russia having the most names in this dataset!

#### -Precision and Recall-
------
Note: Precision and Recall will be computed using sklearn.metrics's ```precision_score``` and ```recall_score``` library.


------

In [ ]:
surname_precision = 0.0
surname_recall = 0.0

In [ ]:
surname_precision = precision_score(y_test, y_pred, average="micro")
surname_recall = recall_score(y_test, y_pred, average="micro")

print("Overall")
print(f"Precision: {surname_precision}")
print(f"Recall: {surname_recall}")

#### -Predictions-
Now we will run some test predictions.

In [ ]:
# let's try predicting a Japanese name
pred_name1 = ["Showa"]
reshape_feature = cv.transform(pred_name1)
surname_model.predict(reshape_feature)

In [ ]:
# Arabic surname name
pred_name2 = ["Bata"]
reshape_feature = cv.transform(pred_name2)
surname_model.predict(reshape_feature)

In [ ]:
# Russian surname name
pred_name3 = ["Jugai"]
reshape_feature = cv.transform(pred_name3)
surname_model.predict(reshape_feature)

In [ ]:
# multiple names (German, English, Japanese, Arabic)
pred_names = ["Samuel", "Drew", "Shunji", "Mustafa"]
reshape_feature = cv.transform(pred_names)
surname_model.predict(reshape_feature)

__Observation 3__: It thinks every name is Russian. This makes sense given Russian surname accuracy is almost 50% while the odds of predicting other nationalities are less than 16%.

---
### Model Developement - Japanese Surnames Only

What if we train the model on only Japanese surnames?

---

#### -Features Exploration-

In [ ]:
# get japanese names
japanese_surnames = surname_df.loc[surname_df["nationality"] == "Japanese"]
japanese_surnames.head()

In [ ]:
features = japanese_surnames["surname"] # features (x) needed to predict nationatlity
target = japanese_surnames["nationality"] # what we are predicting (y)

---
### Train/Test Data

---
To make the data a little more accurate in it's predictions, we are going to split the surnames into train (80%) and test (20%) datasets.

In [ ]:
# vectorize features
cv = CountVectorizer()
X = cv.fit_transform(features)

In [ ]:
# get feature names
# cv.get_feature_names()

In [ ]:
# split the data to train the model
x_train, x_test, y_train, y_test = train_test_split(X, target, test_size=0.35, random_state = 32)

In [ ]:
# fit the model
jap_surname_model = MultinomialNB()
jap_surname_model.fit(x_train, y_train)

In [ ]:
y_pred = jap_surname_model.predict(x_test)

#### -Accuracy-

In [ ]:
# Accuracy using test data
jap_accuracy = f1_score(y_test, y_pred, average='macro')
print(f"f1 score: {jap_accuracy}")

__Observation 1__: Woah. That accuracy. This is solely because I only tested the model using Japanese names.

In [ ]:
# Number of Japanese names in test data
print(y_test.value_counts())

In [ ]:
# number of Japanese names in training data
print(y_train.value_counts())

In [ ]:
# Which nationalities is the model more accurate with
null_accuracy = y_test.value_counts().head(5) / len(y_test)
print(null_accuracy)

In [ ]:
jap_precision = 0.0
jap_recall = 0.0

In [ ]:
jap_precision = precision_score(y_test, y_pred, average="micro")
jap_recall = recall_score(y_test, y_pred, average="micro")

print("Japanese Surnames ")
print(f"Precision: {jap_precision}")
print(f"Recall: {jap_recall}")

#### -Predictions-
Now we will run some test predictions.

In [ ]:
# let's try predicting a Japanese name
pred_name1 = ["Showa"]
reshape_feature = cv.transform(pred_name1)
jap_surname_model.predict(reshape_feature)

In [ ]:
# Arabic surname name
pred_name2 = ["Bata"]
reshape_feature = cv.transform(pred_name2)
jap_surname_model.predict(reshape_feature)

In [ ]:
# Russian surname name
pred_name3 = ["Jugai"]
reshape_feature = cv.transform(pred_name3)
jap_surname_model.predict(reshape_feature)

In [ ]:
# multiple names (German, English, Japanese, Arabic)
pred_names = ["Samuel", "Drew", "Shunji", "Mustafa"]
reshape_feature = cv.transform(pred_names)
jap_surname_model.predict(reshape_feature)

__Observation 2__: The model assumes that all names are Japanese. This is because it has been trained on no other type of name. It is all it knows. This model is biased too biased and has been overfitted.

---
### -Conclusion-

Although the second model has the best accuracy, precision, and recall it is not practical. By training the dataset only on one value, it now assumes that all names are Japanese. While the test data is indeed all Japanese surnames, the model has been overfitted and so there are lots of false positives for names that have not been classified as Japanese. 

We will go with the first model because it is more realistic and practical.

---

In [ ]:
print(f"Accuracy: {surname_accuracy}")
print(f"Precision: {surname_precision}")
print(f"Recall: {surname_recall}")

In [ ]:
print(f"Nationality Accuracy: \n {surname_null_accuracy}")